In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from tqdm.notebook import tqdm
import re
import os
from tqdm import tqdm

In [2]:
processed_dir = "/data1/normantm/angel/sequencing/AP1_816_puro_round2_1_output_NoFilter"
processed_dir1 = "/data1/normantm/angel/sequencing/AP1_816_puro_round2_2_output_NoFilter"

In [3]:
data = pd.read_csv(f"{processed_dir}/processed_filtered_reads_DefaultSample.csv")

In [4]:
data1 = pd.read_csv(f"{processed_dir1}/processed_filtered_reads_DefaultSample.csv")

In [5]:
directory = "/data1/normantm/angel/sequencing/AP1_816_puro_run1_output"
filenames = sorted([entry for entry in os.listdir(directory) if entry.startswith('new_processed') and entry.endswith('.csv')])

In [6]:
identity = []
reporter = []
for f in tqdm(filenames):
    transcript, rep, i = f.rstrip(".csv").split("_")[4:]
    sample = f"{rep}_{i}"
    df = pd.read_csv(f"{directory}/{f}")
    df["sample"] = sample
    if transcript == "identity":
        identity.append(df)
    elif transcript == "reporter":
        reporter.append(df)
    else:
        print("transcript should be either identity or reporter")

identity_df = pd.concat(identity, ignore_index=True)
reporter_df = pd.concat(reporter, ignore_index=True)

100%|██████████████████████████████████████████████| 18/18 [18:38<00:00, 62.12s/it]


In [7]:
directory1 = "/data1/normantm/angel/sequencing/AP1_816_puro_run2_output"
filenames1 = sorted([entry for entry in os.listdir(directory1) if entry.startswith('new_processed') and entry.endswith('.csv')])

In [8]:
identity1 = []
reporter1 = []
for f in tqdm(filenames1):
    transcript, rep, i = f.rstrip(".csv").split("_")[4:]
    sample = f"{rep}_{i}"
    df = pd.read_csv(f"{directory1}/{f}")
    df["sample"] = sample
    if transcript == "identity":
        identity1.append(df)
    elif transcript == "reporter":
        reporter1.append(df)
    else:
        print("transcript should be either identity or reporter")

identity_df1 = pd.concat(identity1, ignore_index=True)
reporter_df1 = pd.concat(reporter1, ignore_index=True)

100%|██████████████████████████████████████████████| 18/18 [20:43<00:00, 69.09s/it]


In [16]:
identity_df = pd.concat([identity_df1, identity_df])
reporter_df = pd.concat([reporter_df1, reporter_df])

In [17]:
data_identity = pd.concat([data, data1]).query("transcript == 'identity'")[identity_df.columns]
data_reporter = pd.concat([data, data1]).query("transcript == 'reporter'")[reporter_df.columns]

In [18]:
combined_identity = pd.concat([identity_df, data_identity], ignore_index=True)
combined_reporter = pd.concat([reporter_df, data_reporter], ignore_index=True)

In [ ]:
#combined_identity.to_csv("/data1/normantm/angel/Pscreen_data/K562_AP1_816_puro/combined_identity_reads.csv", index=False)
#combined_reporter.to_csv("/data1/normantm/angel/Pscreen_data/K562_AP1_816_puro/combined_reporter_reads.csv", index=False)

In [21]:
identity_UMI = combined_identity[["p1_identity", "p2_identity", "sample", "barcode_mapped", "UMI"]].value_counts().to_frame("n")\
.groupby(level=[0, 1, 2, 3]).size().to_frame("UMI")
reporter_UMI = combined_reporter[["p1_identity", "p2_identity", "sample", "barcode_mapped", "UMI"]].value_counts().to_frame("n")\
.groupby(level=[0, 1, 2, 3]).size().to_frame("UMI")

In [22]:
UMI = identity_UMI.merge(reporter_UMI,
                   left_index=True,
                   right_index=True,
                   suffixes=("_identity", "_reporter"), how="outer").fillna(0).reset_index()

In [ ]:
#UMI.to_csv("/data1/normantm/angel/Pscreen_data/K562_AP1_816_puro/subsample_50_UMI_counts.csv", index=False)